# First protocol
_Progresses from top to botttom. Go to bottom and run all for proper use. Modify environment variables and queries as needed Markdown cells will_ _highlight functionality of following code cell. Functionality will be derived from the list below with the numbered list being referenced in other markdown cells._

## Protocol
· Start with keywords:

- Smartchain

- Nft

- Airdrop

- Crypto

- …etc.

1. Sample up to 10k tweets containing at least one term from 100 random hours from the past year (so 1M tweets)

2. Determine the most engaged (top) with users from this combined sample (100 or 1000)

3. Pull up to 1000 comments for each top user

4. Determine top users whose comments mention at least three users other than the top user

5. Expand top user sample if we don’t have at least 100 airdrop seeders

6. Time series chart plots:

7. Top user activity

8. Airdrop seeder activity

9. Negative reaction activity? (based on sentiment analysis of replies to airdrop messages)

10. External crypto value signals (from where?)



# Dependencies
Run the following commands in the terminal to install the required packages

$pip install requests  
$pip install pandas  
$pip install datetime  
$pip install python-dateutil


# Authentication step
In the code cell below replace bearer_token with your bearer token. Run the cell, then delete your bearer token.
This creates the token as an environment variable to be used under the name TOKEN. The token can then be removed so that others do not have access to your token when code is shared via GitHub. The same could be achieved through a .env file and using .gitignore however this would require dotenv package which just makes things more complicaded yada yada just put your token in run the cell and delete it ok thank you.

In [3]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAAMwdgEAAAAA%2BBTpRVpy5ExIM6%2BgaytL9o0%2B63M%3Dt3GdaN5o2Hx03bp0XqrktMAotnw3wGHFH6Z3vYsNmtWbTPUxcF'

In [4]:
"""This cell creates functions to be used for authentication as well as creating endpoints."""
import requests

def auth():
    """Retrieves your bearer token."""
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def create_url(keyword, start_date, end_date, max_results = 10):
    
    #Change to the endpoint you want to collect data from
    search_url = "https://api.twitter.com/2/tweets/search/recent" 

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id',
                    'tweet.fields': 'id,text,author_id',
                    #'user.fields': 'id,name,username,created_at',
                    #'place.fields': 'country',
                    'next_token': {}}
    return (search_url, query_params)

def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

print(auth())

AAAAAAAAAAAAAAAAAAAAAAMwdgEAAAAA%2BBTpRVpy5ExIM6%2BgaytL9o0%2B63M%3Dt3GdaN5o2Hx03bp0XqrktMAotnw3wGHFH6Z3vYsNmtWbTPUxcF


In [45]:
import datetime
import random
"""This code cell contains two functions (is_leap_year and random_date) which help generate a random one hour date range when random_date() is called"""
# Use by calling "start_time, end_time = random_date()"

def is_leap_year(year):
    """Returns True if the given year is a leap year, False otherwise."""
    if year % 4 == 0:
        if year % 100 == 0:
            if year % 400 == 0:
                return True
            else:
                return False
        else:
            return True
    else:
        return False

def random_date():
    """Generate a random one hour date range within the last year in RFC 3339 format to be used with twitter API."""
    year = random.randint(datetime.datetime.now().year - 1, datetime.datetime.now().year)
    month = random.randint(1, datetime.datetime.now().month)
    if month == datetime.datetime.now().month:
        day = random.randint(1, datetime.datetime.now().day)
    elif month == 2:
        if is_leap_year(year):
            day = random.randint(1, 29)
        else:
            day = random.randint(1, 28)
    elif month in [1, 3, 5, 7, 8, 10, 12]:
        day = random.randint(1, 31)
    else:
        day = random.randint(1, 31)
    hour = random.randint(0, 23)
    start_time = datetime.datetime(year, month, day, hour)
    end_time = start_time + datetime.timedelta(hours=1)
    start_timestamp = start_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    end_timestamp = end_time.strftime("%Y-%m-%dT%H:%M:%SZ")
    return start_timestamp, end_timestamp

#start_time1, end_time1 = random_date()
#print(f"The one hour range is from {start_time1} to {end_time1}")

In [6]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
keyword = "Xbox lang:en"
# Choose a random one hour range within the last year
#start_time, end_time = random_date()
start_time = "2022-12-29T17:00:00Z"
end_time = "2022-12-31T20:00:00Z"

max_results = 100

In [25]:
import json
import pandas as pd
import csv

url = create_url(keyword, start_time, end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])
result_dict = json_response["data"]
#while "next_token" in json_response["meta"] and len(result_dict) < 9900:
    #print(json.dumps(json_response, indent=4, sort_keys=True))
    #json_response = connect_to_endpoint(url[0], headers, url[1], json_response["meta"]["next_token"])
    #result_dict.extend(json_response["data"])
#print(json.dumps(json_response, indent=4, sort_keys=True))

with open('data.json', 'w') as f:
    json.dump(result_dict, f)

df = pd.read_json('data.json')

df.head()
#url = create_url(keyword, start_time, end_time, max_results)
#json_response = connect_to_endpoint(url[0], headers, url[1])
#print(json.dumps(json_response, indent=4, sort_keys=True))

Endpoint Response Code: 200


In [ ]:
#Inputs for tweets

start_list = list()
end_list = list()

for _ in range(100):
    start_time, end_time = random_date()
    while start_time in start_list:
        start_time, end_time = random_date()
    start_list.append(start_time)
    end_list.append(end_time)

max_results = 500

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['author id', 'created_at', 'geo', 'id','lang', 'like_count', 'quote_count', 'reply_count','retweet_count','source','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i], max_results)
        json_response = connect_to_endpoint(url[0], headers, url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)

In [51]:
start_list = list()
end_list = list()
for _ in range(0, 100):
    start_time, end_time = random_date()
    print(start_time, end_time)
    print(_)
    while start_time in start_list:
        start_time, end_time = random_date()
    start_list.append(start_time)
    end_list.append(end_time)

print(start_list, end_list)

2023-01-01T07:00:00Z 2023-01-01T08:00:00Z
0
2022-01-02T00:00:00Z 2022-01-02T01:00:00Z
1
2023-01-02T00:00:00Z 2023-01-02T01:00:00Z
2
2022-01-01T17:00:00Z 2022-01-01T18:00:00Z
3
2023-01-02T10:00:00Z 2023-01-02T11:00:00Z
4
2023-01-01T20:00:00Z 2023-01-01T21:00:00Z
5
2023-01-01T04:00:00Z 2023-01-01T05:00:00Z
6
2022-01-02T21:00:00Z 2022-01-02T22:00:00Z
7
2022-01-02T17:00:00Z 2022-01-02T18:00:00Z
8
2023-01-01T16:00:00Z 2023-01-01T17:00:00Z
9
2022-01-02T12:00:00Z 2022-01-02T13:00:00Z
10
2022-01-01T21:00:00Z 2022-01-01T22:00:00Z
11
2022-01-02T14:00:00Z 2022-01-02T15:00:00Z
12
2022-01-02T11:00:00Z 2022-01-02T12:00:00Z
13
2022-01-02T21:00:00Z 2022-01-02T22:00:00Z
14
2022-01-01T03:00:00Z 2022-01-01T04:00:00Z
15
2022-01-01T12:00:00Z 2022-01-01T13:00:00Z
16
2022-01-01T18:00:00Z 2022-01-01T19:00:00Z
17
2023-01-02T02:00:00Z 2023-01-02T03:00:00Z
18
2022-01-02T03:00:00Z 2022-01-02T04:00:00Z
19
2023-01-02T22:00:00Z 2023-01-02T23:00:00Z
20
2022-01-01T03:00:00Z 2022-01-01T04:00:00Z
21
2022-01-02T12:00:00Z

KeyboardInterrupt: 